In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import calibration
import sklearn.linear_model
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
import numpy as np
import scipy.stats
from sklearn.base import clone
from sklearn.model_selection._search import ParameterSampler
from sklearn.metrics import brier_score_loss, make_scorer, log_loss, mean_squared_error
import os
notebook_dir = os.getcwd()
os.chdir(os.path.join(notebook_dir, '..'))
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
# os.chdir('/Users/lucashaupt/Documents/GitHub/nfl-live-win-probability')
from src.generate_data import get_game_data, get_division_data, get_odds_data, get_schedule_data
import statsmodels.api as sm

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [ ]:
df = pd.read_csv("data/yards_gained_all_seasons.csv")
cache = True
game_data = get_game_data(cache=cache)
game_data = game_data.drop_duplicates("game_code")
division_data = get_division_data(cache=cache)
combined_df = df.merge(game_data, on=["game_code", "season"], how="left", copy=False)


In [ ]:
division_data.groupby(["season", "league_id"], as_index=False).count()

In [ ]:
def get_rolling_value(df, freq=16, min_periods=1, shift=1):
    cols = ["total_expected_yards_gained", "total_yards_gained", "total_yards_added", "play_count"]
    df2 = df.rolling(freq, on="game_date", min_periods=min_periods)[cols].mean().shift(shift)
    df4 = pd.concat([df[["game_code", "season", "game_date", "offense_team", "defense_team", "home_team_id", "away_team_id"]],
                     df2[["total_expected_yards_gained", "total_yards_gained", "total_yards_added", "play_count"]]], axis=1)
    return df4


In [ ]:
aggregate_values = ["total_expected_yards_gained", "total_yards_gained", "total_yards_added", "play_count"]
group_values = ["game_code", "season", "game_date", "offense_team", "defense_team", "home_team_id", "away_team_id"]
game_yards = combined_df[group_values + aggregate_values].groupby(group_values, as_index=False).sum()

In [ ]:
rolling_offense = game_yards.sort_values("game_date").groupby("offense_team", as_index=False).apply(get_rolling_value)

In [ ]:
rolling_defense = game_yards.sort_values("game_date").groupby("defense_team", as_index=False).apply(get_rolling_value)

In [ ]:
game_codes = combined_df[["game_code", "home_team_id", "away_team_id", "home_team_abbrev", "away_team_abbrev", "home_score", "away_score", "game_date", "season"]].drop_duplicates()

In [ ]:
game_codes.columns

In [ ]:
rolling_offense.columns

In [ ]:
yards_values = ['total_expected_yards_gained', 'total_yards_gained', 'total_yards_added', 'play_count']
games_home_team = game_codes.merge(rolling_offense[["game_code", "offense_team"] + yards_values], left_on=["game_code", "home_team_id"], right_on=["game_code", "offense_team"], suffixes=["", "_offense"])
games_home_team = games_home_team.merge(rolling_defense[["game_code", "defense_team"] + yards_values], left_on=["game_code", "home_team_id"], right_on=["game_code", "defense_team"], suffixes=["_offense", "_defense"])


In [ ]:
yards_values = ['total_expected_yards_gained', 'total_yards_gained', 'total_yards_added', 'play_count']
games_away_team = game_codes.merge(rolling_offense[["game_code", "offense_team"] + yards_values], left_on=["game_code", "away_team_id"], right_on=["game_code", "offense_team"], suffixes=["", "_offense"])
games_away_team = games_away_team.merge(rolling_defense[["game_code", "defense_team"] + yards_values], left_on=["game_code", "away_team_id"], right_on=["game_code", "defense_team"], suffixes=["_offense", "_defense"])

In [ ]:
games = games_home_team.merge(games_away_team, on=['game_code', 'season', 'home_team_id', 'away_team_id', 'home_team_abbrev', 'away_team_abbrev', 'home_score', 'away_score', 'game_date'], suffixes=["_home", "_away"])
games = games[games["season"]>=2009]


In [ ]:
features = ["total_yards_added_offense_home",
            "total_yards_added_defense_home",
            "total_yards_added_offense_away",
            "total_yards_added_defense_away",]
target = ["home_score","away_score"]

In [ ]:
games[features]

In [ ]:
ridge_model_home = sklearn.linear_model.RidgeCV(alphas=(0.1, 1.0, 10.0))
ridge_model_home.fit(games[features], games["home_score"])
ridge_model_home.score(games[features], games["home_score"])
games["x_home_score"] = ridge_model_home.predict(games[features])

In [ ]:
ridge_model_away = sklearn.linear_model.RidgeCV(alphas=(0.1, 1.0, 10.0))
ridge_model_away.fit(games[features], games["away_score"])
ridge_model_away.score(games[features], games["away_score"])
games["x_away_score"] = ridge_model_away.predict(games[features])

In [ ]:
plt.scatter(games["x_home_score"], games["home_score"])
plt.plot([0,60], [0,60])

In [ ]:
plt.scatter(games["x_away_score"], games["away_score"])
plt.plot([0,60], [0,60])

In [ ]:

X2 = sm.add_constant(games[features])
est = sm.OLS(games["home_score"], X2)
results_new = []
results_fu = est.fit()
for x in np.arange(0.005, 0.25, 0.005).tolist():
    est2 = est.fit_regularized(L1_wt=0, alpha=x)
    results_fr_fit = sm.regression.linear_model.OLSResults(est, 
                                                            est2.params, 
                                                            results_fu.normalized_cov_params)

    results_new.append(results_fr_fit.ssr)
    # print(est2.model.score(est2.params))


In [ ]:
plt.plot(np.arange(0.005, 0.25, 0.005).tolist(), results_new)

In [ ]:
plt.plot(np.arange(0.005, 0.25, 0.005).tolist(), results_new)

In [ ]:
results_fr = est.fit_regularized(L1_wt=0, alpha=0.1, start_params=results_fu.params)
final = sm.regression.linear_model.OLSResults(est, 
                                              results_fr.params, 
                                              est.normalized_cov_params)

print(final.summary())


In [ ]:
games.to_clipboard()

In [ ]:
games["total_yards_added_offense_home_per_play"] = games["total_yards_added_offense_home"] / games["play_count_offense_home"] 
games["total_yards_added_defense_home_per_play"] = games["total_yards_added_defense_home"] / games["play_count_defense_home"] 
games["total_yards_added_offense_away_per_play"] = games["total_yards_added_offense_away"] / games["play_count_offense_away"] 
games["total_yards_added_defense_away_per_play"] = games["total_yards_added_defense_away"] / games["play_count_defense_away"] 


In [ ]:
games.columns

In [ ]:

new_features = [
    'total_yards_added_offense_home_per_play',
    'total_yards_added_defense_home_per_play',
    'total_yards_added_offense_away_per_play',
    'total_yards_added_defense_away_per_play',
    'play_count_offense_home',
    'play_count_defense_home',
    'play_count_offense_away', 
    'play_count_defense_away'

       ]
X2 = sm.add_constant(games[new_features])
est = sm.OLS(games["away_score"], X2)
est2 = est.fit()
print(est2.summary())


In [ ]:

new_features = [
    'total_yards_added_offense_home_per_play',
    'total_yards_added_defense_home_per_play',
    'total_yards_added_offense_away_per_play',
    'total_yards_added_defense_away_per_play',
    'play_count_offense_home',
    'play_count_defense_home',
    'play_count_offense_away', 
    'play_count_defense_away'

       ]
X2 = sm.add_constant(games[new_features])
est = sm.OLS(games["home_score"], X2)
est2 = est.fit()
print(est2.summary())


In [3]:
X2 = sm.add_constant(games[features])
est = sm.OLS(games["away_score"], X2)
est2 = est.fit()
print(est2.summary())


NameError: name 'sm' is not defined

In [ ]:
X2 = sm.add_constant(games[features])
est = sm.OLS(games["home_score"], X2)
est2 = est.fit()
print(est2.summary())


In [ ]:
games["home_team_win"] = np.where(games["home_score"] > games["away_score"], 1, 0)
games["away_team_win"] = np.where(games["home_score"] < games["away_score"], 1, 0)
games["tie"] = np.where(games["home_score"] == games["away_score"], 1, 0)

In [ ]:
X2 = sm.add_constant(games[features])
est = sm.OLS(games["away_score"], X2)
est2 = est.fit()
print(est2.summary())


In [ ]:
import datetime as dt 

schedule = get_schedule_data(cache=cache)
schedule = schedule[schedule["game_date"].dt.date>np.max(pd.to_datetime(games["game_date"]))]


In [ ]:
np.max(pd.to_datetime(games["game_date"]))

In [ ]:
X2 = sm.add_constant(games[features])
est = sm.Logit(games["tie"], X2)
est2 = est.fit()
games["x_tie"] = est2.predict(X2)

print(est2.summary())


In [ ]:
odds = pd.read_parquet("data/odds_data.parquet")
odds = odds.drop_duplicates("game_code")

In [ ]:
games=games.merge(odds[["game_code", "cur_spread"]], on="game_code")

In [ ]:
current_values = games_home_team.append(games_away_team).sort_values("game_date", ascending=False).drop_duplicates(["offense_team"])
current_values = current_values.merge(division_data, left_on=["season", "offense_team"], right_on=["season", "team_id"])

In [ ]:
predictions = schedule.merge(current_values[['offense_team', 'total_yards_added_offense', 'total_yards_added_defense']], left_on=["home_team_id"], right_on=["offense_team"], suffixes=["", "_home"])
predictions = predictions.merge(current_values[['offense_team', 'total_yards_added_offense', 'total_yards_added_defense']], left_on=["away_team_id"], right_on=["offense_team"], suffixes=["_home", "_away"])

In [ ]:
X2 = sm.add_constant(games[features])
est = sm.Logit(games["home_team_win"], X2)
est2 = est.fit()
games["x_home_team_win"] = est2.predict(X2)
predictions["x_home_team_win"] = est2.predict(sm.add_constant(predictions[features]))
print(est2.summary())


In [ ]:
X2 = sm.add_constant(games[features])
est = sm.Logit(games["tie"], X2)
est2 = est.fit()
games["x_tie_calculated"] = est2.predict(X2)
games["x_away_team_win"] = 1 - games["x_tie_calculated"] - games["x_home_team_win"]
predictions["x_tie_calculated"] = est2.predict(sm.add_constant(predictions[features]))
predictions["x_away_team_win"] = 1 - predictions["x_tie_calculated"] - predictions["x_home_team_win"]

print(est2.summary())


In [ ]:
predictions["x_home_score"] = ridge_model_home.predict(predictions[features])
predictions["x_away_score"] = ridge_model_away.predict(predictions[features])
predictions["x_line"] = predictions["x_away_score"] - predictions["x_home_score"]

In [ ]:
models = {}
for team_score in range(int(np.max(games["home_score"]+1))):
    for opponent_score in range(int(np.max(games["away_score"]+1))):
        try:
            models[str(team_score) + str(opponent_score)] = sklearn.linear_model.LogisticRegression()
            models[str(team_score) + str(opponent_score)].fit(games[features], np.where((games["home_score"]==team_score)&(games["away_score"]==opponent_score), 1, 0))
            print(str(team_score) + " - " + str(opponent_score), *models[str(team_score) + str(opponent_score)].intercept_, *models[str(team_score) + str(opponent_score)].coef_)
        except:
            print(str(team_score) + " - " + str(opponent_score))


In [ ]:
game_prediction_matrix = predictions[["game_code"]+features]
game_outcome_matrix = predictions[["game_code"]+features]
list_of_scores = []
list_of_home_wins = []
list_of_ties = []
list_of_away_wins = []
for team_score in range(int(np.max(games["home_score"]+1))):
    for opponent_score in range(int(np.max(games["away_score"]+1))):
        try:
            game_prediction_matrix["score_" + str(team_score) + "_" + str(opponent_score)] = pd.DataFrame(models[str(team_score) + str(opponent_score)].predict_proba(predictions[features]))[1]
            print("score_" + str(team_score) + "_" + str(opponent_score))
            list_of_scores.append("score_" + str(team_score) + "_" + str(opponent_score))
            if team_score > opponent_score:
                list_of_home_wins.append("score_" + str(team_score) + "_" + str(opponent_score))
            elif team_score == opponent_score:
                list_of_ties.append("score_" + str(team_score) + "_" + str(opponent_score))
            else:
                list_of_away_wins.append("score_" + str(team_score) + "_" + str(opponent_score))

        except:
            game_prediction_matrix["score_" + str(team_score) + "_" + str(opponent_score)] = 0
            list_of_scores.append("score_" + str(team_score) + "_" + str(opponent_score))



In [ ]:
game_exact_score_predictions = []
for game_code in predictions["game_code"]:
    game_dict = {
        "game_code": game_code,
        "home_team_id": predictions[predictions["game_code"] == game_code][
            "home_team_id"
        ].item(),
        "away_team_id": predictions[predictions["game_code"] == game_code][
            "away_team_id"
        ].item(),
    }
    game_prediction = game_prediction_matrix[
        game_prediction_matrix["game_code"] == game_code
    ][list_of_scores].values[0]
    prob_adjustment = np.sum(game_prediction_matrix[game_prediction_matrix["game_code"]==game_code][list_of_scores].values[0])
    game_dict["pred_exact_score"] = (game_prediction / np.sum(game_prediction)).tolist()
    game_dict["pred_outcome"] = [
        np.sum(
            game_prediction_matrix[game_prediction_matrix["game_code"] == game_code][
                list_of_home_wins
            ].values[0]
        )
        / prob_adjustment,
        np.sum(
            game_prediction_matrix[game_prediction_matrix["game_code"] == game_code][
                list_of_ties
            ].values[0]
        )
        / prob_adjustment,
        np.sum(
            game_prediction_matrix[game_prediction_matrix["game_code"] == game_code][
                list_of_away_wins
            ].values[0]
        )
        / prob_adjustment,
    ]

    game_dict["current_score"] = [0, 0]
    game_exact_score_predictions.append(game_dict)


In [ ]:
results_df = games.loc[games.season==2021, ["game_code", "home_team_id", "away_team_id", "home_score", "away_score"]].to_dict(orient='records')


In [ ]:
team_names = games.loc[games.season==2021, ["home_team_id", "home_team_abbrev"]]
current_division_data = current_values[["offense_team", "conference_id", "division_id"]].rename(columns={'offense_team':'home_team_id'})
team_names= team_names.merge(current_division_data)
team_names = team_names.rename(columns={'home_team_id': 'id', 'home_team_abbrev': 'name'}).drop_duplicates().to_dict(orient='records')

In [ ]:
sim_data = {
    "teams": team_names,
    "results": results_df,
    "predictions": game_exact_score_predictions,
    "prediction_params": {"max_home_score": 57, "max_away_score": 59}

}
import json
data_dir = "data/"
with open(os.path.join(data_dir, 'simulation_inputs.json'), 'w') as f:
    json.dump(sim_data, f)

In [ ]:
sim_data["predictions"][0]["pred_exact_score"]

In [ ]:
sim_data["teams"]

In [ ]:
sim_data["predictions"][0]

In [ ]:
home_scores = []
away_scores = []

for team_score in range(int(np.max(games["home_score"]+1))):
    for opponent_score in range(int(np.max(games["away_score"]+1))):
        home_scores.append(team_score)
        away_scores.append(opponent_score)
score_game_example = pd.DataFrame([list_of_scores, (sim_data["predictions"][0]["pred_exact_score"]), home_scores, away_scores]).T
score_game_example=score_game_example.rename(columns={1: "probability", 2: "home_score", 3: "away_score"})



In [ ]:
import seaborn as sns
from matplotlib.colors import LogNorm, Normalize
data = pd.DataFrame(score_game_example[["home_score", "away_score", "probability"]], dtype="float")
data = data.pivot("home_score", "away_score", "probability")
data.head()

sns.heatmap(data, square=True)

In [ ]:
score_game_example_home = score_game_example.groupby("home_score").sum()
score_game_example_away = score_game_example.groupby("away_score").sum()
plt.plot(score_game_example_home["probability"])
plt.plot(score_game_example_away["probability"])


In [ ]:
pd.concat([games[["game_code", "x_home_team_win", "x_away_team_win", "game_date"]], predictions[["game_code", "x_home_team_win", "x_away_team_win", "game_date"]]]).rename(columns={"x_home_team_win": "prior_home", "x_away_team_win": "prior_away"}).to_csv("data/game_priors.csv")
# os.system('say "done notebook"')
